In [6]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

whiteboard = np.ones((480, 640, 3), dtype=np.uint8) * 255

prev_x, prev_y = None, None

def count_fingers(hand_landmarks):
    tips_ids = [8, 12, 16, 20]
    count = 0

    if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
        count += 1


    for tip_id in tips_ids:
        if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[tip_id - 2].y:
            count += 1

    return count

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (640, 480))
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            
            fingers_up = count_fingers(hand_landmarks)

            
            h, w, _ = frame.shape
            index_finger_tip = hand_landmarks.landmark[8]
            cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

            
            if fingers_up == 0:
                prev_x, prev_y = None, None
                cv2.putText(frame, "Drawing stopped", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            else:
                
                if prev_x is None:
                    prev_x, prev_y = cx, cy

                cv2.line(whiteboard, (prev_x, prev_y), (cx, cy), (0, 0, 0), 5)

                prev_x, prev_y = cx, cy

    else:
       
        prev_x, prev_y = None, None

    
    cv2.imshow("Camera", frame)
    cv2.imshow("Whiteboard", whiteboard)

    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
